<a href="https://www.kaggle.com/code/programgeek01/us-tweet-analysis?scriptVersionId=92758657" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/us-politician-tweets/tweets.csv
/kaggle/input/us-politician-tweets/__results__.html
/kaggle/input/us-politician-tweets/politics_tweets.zip
/kaggle/input/us-politician-tweets/__notebook_source__.ipynb
/kaggle/input/us-politician-tweets/__notebook__.ipynb
/kaggle/input/us-politician-tweets/__output__.json
/kaggle/input/us-politician-tweets/custom.css
/kaggle/input/us-politicians-twitter-dataset/dataset.csv


In [2]:
tweets = pd.read_csv("/kaggle/input/us-politician-tweets/politics_tweets.zip")
tweets.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,20,21,25,26,27,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1454437376403644425,1454437372725190664,1635599867000.0,2021-10-30 13:17:47,0.0,NaN,If your sample ballot never arrived or you are...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1454437374839119879,1454437372725190664,1635599866000.0,2021-10-30 13:17:46,0.0,NaN,Today is the last day for early voting! You ca...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1454437372725190664,1454437372725190664,1635599866000.0,2021-10-30 13:17:46,0.0,NaN,Virginia! 🚨 Our democracy is strongest when we...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1454144784159891466,1454144784159891466,1635530107000.0,2021-10-29 17:55:07,0.0,NaN,ICYMI: I joined @LCVoters &amp; other climate ...,en,['buildbackbatter'],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1454112667996758020,1454112667996758020,1635522450000.0,2021-10-29 15:47:30,0.0,NaN,TODAY is the last day to nominate a VA-04 vete...,en,[],[],...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [3]:
tweets = tweets[["id", "username", "tweet"]]
tweets = tweets[[isinstance(x, str) for x in tweets["username"]]]

In [4]:
politicians = list(set(tweets["username"]))

In [5]:
network_graph = {'Source': [], 'Target': [], 'Type': [], 'weight': []}
network_graph = pd.DataFrame(data=network_graph)

In [6]:
!pip install flair
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
def flair_prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    if "POSITIVE" in str(score):
        return "pos"
    elif "NEGATIVE" in str(score):
        return "neg"
    else:
        return "neu"

     |████████████████████████████████| 401 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 981 kB 60.2 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 4.7 MB/s  eta 0:00:01
     |████████████████████████████████| 788 kB 60.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 57.0 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 1.9 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 199 kB 63.8 MB/s eta 0:00:01
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9691 sha256=3997c4183f9ac4043ed58c5f3802f853d9535401bafb8b2f8b75b82fddb9af0c
  Stored in directory: /root/.cache/pip/wheels/ba/e0/7e/726e872a53f7358b4b96a9975b04e98113b005cd8609a63abc
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=11

100%|██████████| 265512723/265512723 [00:11<00:00, 22831138.76B/s]

2022-04-11 22:30:33,092 copying /tmp/tmpnays6jpa to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2022-04-11 22:30:33,387 removing temp file /tmp/tmpnays6jpa
2022-04-11 22:30:33,424 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [7]:
for row in tqdm(tweets.iterrows(),total=tweets.shape[0]):
    row = row[1]
    polarity = None
    for pol in politicians:
        if pol in row["tweet"]:
            if polarity == None:
                polarity = -1 if flair_prediction(row["tweet"]) == "neg" else 1
            queried = network_graph.loc[(network_graph["Source"] == "me") & (network_graph["Target"] == "you")]
            if len(queried) == 0:
                network_graph = network_graph.append({'Source': row["username"], 'Target': pol, 'Type': "Undirected", 'weight': polarity}, ignore_index=True)
            else:
                network_graph.at[queried.index[0],"weight"]+=polarity

100%|██████████| 68279/68279 [07:50<00:00, 145.17it/s]


In [8]:
network_graph.head()
network_graph.to_csv("adjacency_graph.csv", index=False)

In [9]:
network_graph = pd.read_csv("adjacency_graph.csv")

In [10]:
network_graph = network_graph[network_graph["Target"] != "POTUS"]
network_graph = network_graph[network_graph["Target"] != "VP"]

In [11]:
import networkx as nx
G = nx.from_pandas_edgelist(network_graph, source="Source", target="Target", edge_attr="weight")

In [12]:
usernames = pd.read_csv("/kaggle/input/us-politicians-twitter-dataset/dataset.csv")
usernames = usernames[["Twitter_username","Political_party"]]

In [13]:
usernames = usernames.set_index('Twitter_username').to_dict()["Political_party"]
parties = usernames.copy()

In [14]:
for i in usernames:
    if "democratic" in usernames[i].lower():
        usernames[i] = 0
    else:
        usernames[i] = 1

### Base Pyvis Visualization

In [15]:
!pip install pyvis
from pyvis.network import Network

In [16]:
net = Network(notebook=True)
net.from_nx(G)
net.save_graph("graph.html")
net.show('graph.html')

### Bokeh Weighted Visualization

In [48]:
!pip install bokeh
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.transform import linear_cmap
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8, RdBu8, turbo, viridis
import matplotlib.cm as cm
output_notebook()

Loading BokehJS ...

In [18]:
degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees)

In [19]:
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(G)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
nx.set_node_attributes(G, name='party_name', values=usernames)
nx.set_node_attributes(G, name='party_string', values=parties)

In [20]:
for i in G.nodes():
    if i not in usernames:
        usernames[i] = 0
usernames["rosadelauro"] = 10

In [21]:
import community
parts = community.best_partition(G, partition=usernames, weight='None')
values = [(node,parts.get(node)) for node in G.nodes()]

In [22]:
nx.set_node_attributes(G, name='communities', values=dict(values))

In [41]:
turbo

<function bokeh.palettes.turbo(n: 'int') -> 'Palette'>

In [52]:
maxval = max(parts.values())
radii = np.random.random(size=maxval + 1) * 1.5

fcm = linear_cmap('communities', palette=viridis(maxval+1),low=0,high=maxval)

In [53]:
#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'party_name'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = RdBu8

#Choose a title!
title = 'US Politics Graph'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Person", "@index"),
        ("Mentions", "@degree"),
    ("Real Party", "@party_string")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html\
network_vis = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0#min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = 1#max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
cmap = cm.get_cmap('viridis', max(parts.values()) + 1)
network_vis.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=fcm)

#Set edge opacity and width
network_vis.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_vis)

show(plot)
save(plot, filename=f"{title}.html")

'/kaggle/working/US Politics Graph.html'